In [53]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown


In [54]:
myclient = pymongo.MongoClient(config.MONGO_URI)
stagingDb = myclient[config.DB_STAGING]
analyseDb = myclient[config.DB_ANALYSE]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
AIRFLOW_WASSTRAAT_CONFIG_NEW = "./wasstraat_config/Wasstraat_Config_Harmonize_new.xlsx"

SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie_v2.xlsx"

In [55]:
grp_aggr = [{"$match" : {'brondata.table': "ARTEFACT"}}, {"$addFields": {'vondstnr': "$brondata.VONDST"}}]

df_artefact = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
df_artefact.head(5)

,_id,brondata,projectcd,aantal,artefactnr,gewicht,materiaalcode,materiaalspecifiek,vondstnr,soort,key,key_project,key_vondst
0,62222cab8082b2c0671db3c9,"{'_id': 62222cab8082b2c0671db3c9, 'ARTEFACT': ...",DC024,11.0,2.0,0.0,OPH,-,13,Artefact,PDC024V13A2,PDC024,PDC024V13
1,62222cab8082b2c0671db3ca,"{'_id': 62222cab8082b2c0671db3ca, 'ARTEFACT': ...",DC024,3.0,6.0,0.0,OPH,-,15,Artefact,PDC024V15A6,PDC024,PDC024V15
2,62222cab8082b2c0671db3cb,"{'_id': 62222cab8082b2c0671db3cb, 'ARTEFACT': ...",DC024,3.0,8.0,0.0,OPH,-,21,Artefact,PDC024V21A8,PDC024,PDC024V21
3,62222cab8082b2c0671db3cc,"{'_id': 62222cab8082b2c0671db3cc, 'ARTEFACT': ...",DC024,28.0,9.0,0.0,PYP,-,1,Artefact,PDC024V1A9,PDC024,PDC024V1
4,62222cab8082b2c0671db3cd,"{'_id': 62222cab8082b2c0671db3cd, 'ARTEFACT': ...",DC024,1.0,10.0,0.0,PYP,-,14,Artefact,PDC024V14A10,PDC024,PDC024V14


In [56]:
## updates=[ UpdateOne({'_id':x['_id']}, {'$set':x}, upsert=True) for x in [v.dropna().to_dict() for k,v in df.iterrows()]]  # 


updates = []
for i, row in df_artefact.iterrows():
    updates.append(pymongo.UpdateMany({'projectcd': row['projectcd'], 'artefactnr': row['artefactnr']}, 
        {'$set': {'aantal': row['aantal'], 
                  'gewicht': row['gewicht'], 
                  'materiaalcode': row['materiaalcode'], 
                  'materiaalspecifiek': row['materiaalspecifiek'],
                  'vondstnr': None}}))
    
analyseCol.bulk_write(updates)

In [57]:
grp_aggr = [{"$match" : {"brondata.ARTEFACT": {'$exists': {"$toBool": 1}}}},
            {"$match" : {"brondata.table": { "$ne": "ARTEFACT" }}}]

df_match = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
df_match.head(5)

,_id,brondata,projectcd,aantal,artefactnr,gewicht,opmerkingen,subnr,tekeningnr,splitid,...,oor_steel,oppervlakte,productiewijze,randdiameter,randindex,randpercentage,subbaksel,type_rand,vorm,wanddikte
0,62222cec1f63db9f28033cb7,"{'_id': 62222cec1f63db9f28033cb7, 'ARTEFACT': ...",DC179,4.0,142.0,0.0,-,1.0,-,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62222cec1f63db9f28033cb8,"{'_id': 62222cec1f63db9f28033cb8, 'ARTEFACT': ...",DC179,4.0,142.0,0.0,-,2.0,-,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62222cec1f63db9f28033cb9,"{'_id': 62222cec1f63db9f28033cb9, 'ARTEFACT': ...",DC179,4.0,142.0,0.0,-,3.0,-,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62222cec1f63db9f28033cba,"{'_id': 62222cec1f63db9f28033cba, 'ARTEFACT': ...",DC179,4.0,142.0,0.0,-,4.0,-,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62222cdef644ed1791e0466e,"{'_id': 62222cdef644ed1791e0466e, 'ARTEFACT': ...",DC154,3.0,33.0,0.0,-,1.0,-,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
grp_aggr = [
{"$match": {"soort": "Artefact"}},
{"$match": {"$and": [{"subnr": {"$exists": True}}, {"brondata.ARTEFACT": {"$exists": False}}, {"brondata.table": {"$nin": ["ROMEINS AARDEWERK"]}}, {"projectcd": {"$nin": ["DC112"]}}] }},
{"$replaceRoot": {"newRoot": {"$arrayToObject": { "$filter" : { "input" : { "$objectToArray" : "$$ROOT"}, "as" : "item", "cond" : { "$and" : [{ "$ne" : ["$$item.v", np.nan]}, { "$ne" : ["$$item.v", None]},{ "$ne" : ["$$item.v", '-']}]}}}}}},
{"$group": {'_id': {'key_subnr': '$key_subnr', 'artefactsoort': '$artefactsoort'},'doc': {'$mergeObjects': '$$ROOT'},'brondata': {'$addToSet': '$$ROOT.brondata'},'tables': {'$addToSet': '$$ROOT.brondata.table'},'projects': {'$addToSet': '$$ROOT.brondata.project'}}},
{"$project": {"doc.brondata": 0}},
{"$addFields": { "doc.brondata": "$brondata", "doc.wasstraat.tables": "$tables", "doc.wasstraat.projects": "$projects" }},
{"$replaceRoot": {"newRoot": '$doc'}},
{"$addFields": {"brondata_count": { "$size": "$brondata" }}}
]

df_artefact = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
df_artefact.head(5)
grp_aggr

[{'$match': {'soort': 'Artefact'}},
 {'$match': {'$and': [{'subnr': {'$exists': True}},
    {'brondata.ARTEFACT': {'$exists': False}},
    {'brondata.table': {'$nin': ['ROMEINS AARDEWERK']}},
    {'projectcd': {'$nin': ['DC112']}}]}},
 {'$replaceRoot': {'newRoot': {'$arrayToObject': {'$filter': {'input': {'$objectToArray': '$$ROOT'},
      'as': 'item',
      'cond': {'$and': [{'$ne': ['$$item.v', nan]},
        {'$ne': ['$$item.v', None]},
        {'$ne': ['$$item.v', '-']}]}}}}}},
 {'$group': {'_id': {'key_subnr': '$key_subnr',
    'artefactsoort': '$artefactsoort'},
   'doc': {'$mergeObjects': '$$ROOT'},
   'brondata': {'$addToSet': '$$ROOT.brondata'},
   'tables': {'$addToSet': '$$ROOT.brondata.table'},
   'projects': {'$addToSet': '$$ROOT.brondata.project'}}},
 {'$project': {'doc.brondata': 0}},
 {'$addFields': {'doc.brondata': '$brondata',
   'doc.wasstraat.tables': '$tables',
   'doc.wasstraat.projects': '$projects'}},
 {'$replaceRoot': {'newRoot': '$doc'}},
 {'$addFields': {'br

In [59]:
len(df_artefact)

4660

In [60]:
inverse_match = {"$match": {"$or": [{"subnr": {"$exists": False}}, {"brondata.ARTEFACT": {"$exists": True}}, {"brondata.table": {"$in": ["ROMEINS AARDEWERK"]}}, {"projectcd": {"$in": ["DC112"]}}] }}
grp_aggr = [{"$match": {"soort": "Artefact"}}, inverse_match]
df_artefact = pd.DataFrame(list(analyseCol.aggregate(grp_aggr)))
df_artefact.head(5)

{"$and": [{"soort": "Artefact"}, {"brondata.table": {"$not": "ARTEFACT"}}, inverse_match["$match"]]}

{'$and': [{'soort': 'Artefact'},
  {'brondata.table': {'$not': 'ARTEFACT'}},
  {'$or': [{'subnr': {'$exists': False}},
    {'brondata.ARTEFACT': {'$exists': True}},
    {'brondata.table': {'$in': ['ROMEINS AARDEWERK']}},
    {'projectcd': {'$in': ['DC112']}}]}]}

In [61]:
len(df_artefact)

10150

In [62]:
df_artefact = pd.DataFrame(list(analyseCol.find({"soort": "Artefact"})))
df_artefact.head(5)



,_id,brondata,projectcd,afmetingen,conservering,doosnr,fotonr,functievoorwerp,materiaalcode,putnr,...,oriëntatie,pathologie,skeletdeel,slijtage,slijtage_onderkaaks_DP4,slijtage_onderkaaks_M1,slijtage_onderkaaks_M2,slijtage_onderkaaks_M3,symmetrie,vergroeiing
0,62222c85cdf377c8f352b2e0,"{'_id': 62222c85cdf377c8f352b2e0, 'ID': 6, 'CO...",DC005,53 x 62 x 3,0.0,790.0,DC05_P01_H0001_734_1 t/m 4.jpg,Toiletartikel,hoorn,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62222c85cdf377c8f352b2e1,"{'_id': 62222c85cdf377c8f352b2e1, 'ID': 7, 'CO...",DC005,27 x 75 x 2,0.0,790.0,DC05_P01_H0001_731_1 t/m 3.jpg,Toiletartikel,hoorn,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62222c85cdf377c8f352b2e2,"{'_id': 62222c85cdf377c8f352b2e2, 'ID': 8, 'CO...",DC005,108 x 72 x 2,0.0,790.0,DC05_P01_H0001_730_1 t/m 5.jpg,Toiletartikel,hoorn,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62222c85cdf377c8f352b2e3,"{'_id': 62222c85cdf377c8f352b2e3, 'ID': 9, 'CO...",DC005,114 x 80 x 3,0.0,790.0,DC05_P01_H0001_733_1 t/m 4.jpg,Toiletartikel,hoorn,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62222c85cdf377c8f352b2e4,"{'_id': 62222c85cdf377c8f352b2e4, 'ID': 10, 'C...",DC005,65 x 75 x 3,0.0,790.0,DC05_P01_H0001_735_1 t/m 4.jpg,Toiletartikel,hoorn,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
len(df_artefact)

15135

In [122]:
import copy

ARTEFACT_NOT_MERGE_PROJECTS = ['DC112']
ARTEFACT_NOT_MERGE_TABLES = ['ROMEINS AARDEWERK']
AGGR_PHASE_CLEAN_EMPTY = {'$replaceRoot': {'newRoot': {'$arrayToObject': {'$filter': {'input': {'$objectToArray': '$$ROOT'},
      'as': 'item',
      'cond': {'$and': [{'$ne': ['$$item.v', np.nan]},
        {'$ne': ['$$item.v', None]},
        {'$ne': ['$$item.v', np.nan]},
        {'$ne': ['$$item.v', '-']}]}}}}}}

AGGREGATE_MOVE = [
    {'$match': { 'soort': "XXX" } },
    {'$addFields': {'wasstraat.tables': ['$brondata.table'],
        'wasstraat.projects': ['$projectcd'], 
        'brondata': ['$brondata']}},
    { "$merge": { "into": { "db": config.DB_ANALYSE, "coll": config.COLL_ANALYSE_CLEAN }, "on": "_id",  "whenMatched": "replace", "whenNotMatched": "insert" } }
    ]

AGGREGATE_MERGE = [{'$match': {'soort': 'XXX'}},
 {'$group': {
    'doc': {'$mergeObjects': '$$ROOT'},
    'brondata': {'$addToSet': '$$ROOT.brondata'},
    'tables': {'$addToSet': '$$ROOT.brondata.table'},
    'projects': {'$addToSet': '$$ROOT.brondata.project'}}},
 {'$project': {'doc.brondata': 0}},
 {'$addFields': {'doc.brondata': '$brondata',
    'doc.wasstraat.tables': '$tables',
    'doc.wasstraat.projects': '$projects'}},
 {'$replaceRoot': {'newRoot': '$doc'}},
 {'$addFields': {'brondata_count': {'$size': '$brondata'}}},
 { "$merge": { "into": { "db": config.DB_ANALYSE, "coll": config.COLL_ANALYSE_CLEAN }, "on": "_id",  "whenMatched": "replace", "whenNotMatched": "insert" } }
]

INVERSE_MATCH_ARTEFACT = {"$match": {'$and': [{'soort': 'Artefact'},
  {'brondata.table': {'$nin': ['ARTEFACT']}},
  {'$or': [{'subnr': {'$exists': False}},
    {'brondata.ARTEFACT': {'$exists': True}},
    {'brondata.table': {'$in': ARTEFACT_NOT_MERGE_TABLES}},
    {'projectcd': {'$in': ARTEFACT_NOT_MERGE_PROJECTS}}]}]}}


def getMoveAggregate(soort):
    aggr = copy.deepcopy(AGGREGATE_MOVE)
    aggr.insert(1, AGGR_PHASE_CLEAN_EMPTY)
    if soort == 'Artefact':
        aggr.insert(1, INVERSE_MATCH_ARTEFACT)
        
    aggr[0]['$match']['soort'] = soort
    return aggr

def getMergeAggregate(soort, key='key'):
    aggr = copy.deepcopy(AGGREGATE_MERGE)
    aggr.insert(1, AGGR_PHASE_CLEAN_EMPTY)
    aggr[2]["$group"]["_id"] = {key: "$"+key}

    if soort == 'Artefact':
        aggr.insert(1, 
            {'$match': {'$and': [{'subnr': {'$exists': True}},
                {'brondata.ARTEFACT': {'$exists': False}},
                {'brondata.table': {'$nin': ARTEFACT_NOT_MERGE_TABLES}},
                {'projectcd': {'$nin': ARTEFACT_NOT_MERGE_PROJECTS}}]}})
        aggr[3]["$group"]["_id"]['artefactsoort'] = "$artefactsoort"
    aggr[0]['$match']['soort'] = soort

    return aggr

aggr = getMergeAggregate('Artefact', key="key_subnr")
#aggr = getMergeAggregate('Vondst')
#aggr = getMoveAggregate('Vondst')
aggr

[{'$match': {'soort': 'Artefact'}},
 {'$match': {'$and': [{'subnr': {'$exists': True}},
    {'brondata.ARTEFACT': {'$exists': False}},
    {'brondata.table': {'$nin': ['ROMEINS AARDEWERK']}},
    {'projectcd': {'$nin': ['DC112']}}]}},
 {'$replaceRoot': {'newRoot': {'$arrayToObject': {'$filter': {'input': {'$objectToArray': '$$ROOT'},
      'as': 'item',
      'cond': {'$and': [{'$ne': ['$$item.v', nan]},
        {'$ne': ['$$item.v', None]},
        {'$ne': ['$$item.v', nan]},
        {'$ne': ['$$item.v', '-']}]}}}}}},
 {'$group': {'doc': {'$mergeObjects': '$$ROOT'},
   'brondata': {'$addToSet': '$$ROOT.brondata'},
   'tables': {'$addToSet': '$$ROOT.brondata.table'},
   'projects': {'$addToSet': '$$ROOT.brondata.project'},
   '_id': {'key_subnr': '$key_subnr', 'artefactsoort': '$artefactsoort'}}},
 {'$project': {'doc.brondata': 0}},
 {'$addFields': {'doc.brondata': '$brondata',
   'doc.wasstraat.tables': '$tables',
   'doc.wasstraat.projects': '$projects'}},
 {'$replaceRoot': {'newRoot'

In [123]:
aggr.pop()

pd.DataFrame(list(analyseCol.aggregate(aggr)))

,_id,brondata,projectcd,datering,literatuur,putnr,subnr,vondstnr,soort,artefactsoort,...,determinatieniveau,autoriteit,verbinding,puntlengte,bakseltype,past_aan_andere_nummers,vlaknr,geconserveerd,percentage,ontwerper
0,62222ca2a9bddbff5983273b,"[{'_id': 62222ca2a9bddbff5983273b, 'ID': 46, '...",DC021,1790,krommenhoek blz. 318,2.0,38.0,1.0,Artefact,Kleipijp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62222c9e82d3d502cacdd7e6,"[{'_id': 62222c9e82d3d502cacdd7e6, 'ID': 17, '...",DC020,1730,NaN,1.0,5.0,5.0,Artefact,Kleipijp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62222d113778c829b5414b29,"[{'_id': 62222d113778c829b5414b29, 'ID': 172, ...",DC097,1500-1525,NaN,1.0,2172.0,304.0,Artefact,Glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62222cd64e88e949bf609283,"[{'_id': 62222cd64e88e949bf609283, 'ID': 418, ...",DC033,1500-1600,NaN,5.0,29.0,1.0,Artefact,Aardewerk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62222c7ee9dc046c5e905394,"[{'_id': 62222c7ee9dc046c5e905394, 'ID': 195, ...",DB034,NaN,NaN,10.0,701.0,1071.0,Artefact,Hout,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4655,62222d1088c76cb053b748f0,"[{'_id': 62222d1088c76cb053b748f0, 'ID': 105, ...",DC097,NaN,NaN,1.0,802.0,299.0,Artefact,Leer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4656,62222cd64e88e949bf6091f5,"[{'_id': 62222cd64e88e949bf6091f5, 'ID': 276, ...",DC033,NaN,NaN,4.0,135.0,17.0,Artefact,Aardewerk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4657,62222cbe3ca8946c293a8f94,"[{'_id': 62222cbe3ca8946c293a8f94, 'ID': 140, ...",DC027,1660,NaN,1.0,88.0,90.0,Artefact,Kleipijp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4658,62222d0456796d2a3e743927,"[{'_id': 62222d0456796d2a3e743927, 'ID': 93, '...",DC036,1500-1800,NaN,1.0,91.0,9.0,Artefact,Aardewerk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
